In [102]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.1 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=0e9924d20c0d3dbf56be061405fe5082e4ff2b3176379f761152edf2a0598831
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [75]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [1]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [79]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [80]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

[Errno 2] No such file or directory: 'gdrive/MyDrive/2021-online-language-offline-impact-main'
/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [83]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0605.csv		       Github
 a0608.csv		       heatmap529_UK.ipynb
 a0609.csv		       heatmap530.ipynb
 ccc_compiled.csv	       Heat_Map_Online_language.ipynb
 csv			       __MACOSX
 csv.zip		       mapping_UK
 dataframe_csv		       mapping_US
 dataframe_csv_EU	       non_violant1.csv
 dataframe_csv_UK	       non_violant2.csv
 dataframe_result_529_UK.csv   UK
 dataframe_result_530_UK.csv   violant.csv
 dataframe_result_531_UK.csv  '新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_result_csv


In [76]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
ct_path_601 = 'csv/US05_30.csv'

In [96]:
ct = pd.read_csv(ct_path_601)

In [ ]:
ct[:100]

In [ ]:
ct.shape[1]

In [ ]:
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [ ]:
len(columns2)

In [ ]:
ct.columns=columns2

In [ ]:
ct

In [ ]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [ ]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [ ]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [ ]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [98]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [99]:
###
ct_ner = ct

In [100]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Spartanburg': 1},{'Spartanburg': 1},{'Spartanburg': 1}
1,"{'US': 1, 'Minnesota': 1}","{'US': 1, 'Minnesota': 1}",{'Minnesota': 1}
2,{'Jacksonville': 1},"{'JACKSONVILLE': 1, 'Fla.': 1, 'Jacksonville':...",{'Jacksonville': 1}
3,{},{},{}
4,{},{'Fargo': 2},{}


In [101]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

TypeError: ignored

In [ ]:
ct_ner['NER-msg']

In [ ]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [ ]:
ct_ner['NER']

0                  [Spartanburg, Spartanburg, Spartanburg]
1                [Minnesota, US, Minnesota, US, Minnesota]
2        [Jacksonville, Jacksonville, Minneapolis, JACK...
3                                                       []
4                                                  [Fargo]
                               ...                        
19648                         [New York City, Minneapolis]
19649                           [Los Angeles, Minneapolis]
19650                                            [Atlanta]
19651                      [Minneapolis, Denver, Colorado]
19652           [Oakland, San Jose, Minneapolis, San Jose]
Name: NER, Length: 19653, dtype: object

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [ ]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg]
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]"
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1670764379911054/post...,1516,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,2,0,1,3,0,0,5917

In [ ]:
ct_ner['NER-1']=ct_ner['NER']

In [ ]:
ct_ner['NER-1'][0]

['spartanburg']

In [ ]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg],[spartanburg]
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]","[minnesota, us]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]","[jacksonville, minneapolis, jacksonville, fla.]"
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.co

In [7]:
%cd mapping_US

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_US


In [8]:
!ls

dataframe_mapping_0529.csv  dataframe_mapping_0606.csv
dataframe_mapping_0530.csv  dataframe_mapping_0607.csv
dataframe_mapping_0531.csv  dataframe_mapping_0608.csv
dataframe_mapping_0601.csv  dataframe_mapping_0609.csv
dataframe_mapping_0602.csv  dataframe_mapping_0610.csv
dataframe_mapping_0603.csv  dataframe_mapping_0611.csv
dataframe_mapping_0604.csv  datarframe_post
dataframe_mapping_0605.csv  post_specific_mapping_US.zip


In [ ]:
ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [ ]:
len(ct_mapping['City'])

1791

In [ ]:
len(ct_ner['NER-1'])

19653

In [ ]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,[Spartanburg],[Spartanburg],[Spartanburg],[spartanburg],"[Spartanburg, Spartanburg County, South Caroli..."
1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"[Minnesota, US]","[Minnesota, US]",[Minnesota],"[minnesota, us]","[Minnesota, United States, United States]"
2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national ‘I...,newsbreak.com,"JACKSONVILLE, Fla. – People in Jacksonville jo...",Demonstrators in Jacksonville join national ‘I...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,[Jacksonville],"[Jacksonville, Minneapolis, JACKSONVILLE, Fla.]",[Jacksonville],"[jacksonville, minneapolis, jacksonville, fla.]","[Jacksonville, Duval County, Florida, United S..."
3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[],[]
4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Support for George Floyd in Far...,newsbreak.com,"FARGO, ND – A crowd in the thousands showed th...",Thousands Show Support for George Floyd in Far...,1.0,https:/

In [ ]:
ct_ner['NER-1'][12]

['Elmira, Chemung County, New York, 14904-1306, United States',
 'Minneapolis, Hennepin County, Minnesota, 55415:55454, United States']

In [ ]:
ct_ner.to_csv('dataframe0530_post.csv')

In [63]:
df_post=pd.read_csv('dataframe0530_post.csv',encoding='ISO-8859-1')

In [68]:
df_post['NER-2']=df_post['NER']

In [69]:
df_post[:100]

,Unnamed: 0,Unnamed: 0.1,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1,NER-2
0,0,0,1717735001821003_2576251915969303,Facebook,2020-05-30 22:01:36,2021-07-27 06:27:54,link,Spartanburg protesters show support for George...,newsbreak.com,The image of a black man held down and taking ...,Spartanburg protesters show support for George...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1717735001821003/post...,1228,-0.875000,1.0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,1,0,2,1,0,0,9975243,"Gaffney, SC Local - News Break",GaffneySCLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-26 19:49:48,en,0,9975243|2576251915969303,['Spartanburg'],['Spartanburg'],['Spartanburg'],['spartanburg'],"['Spartanburg, Spartanburg County, South Carol...",['spartanburg']
1,1,1,122163047886_10157592533682887,Facebook,2020-05-30 22:01:35,2021-08-31 06:14:48,link,Military puts police units on alert for possib...,actionnewsnow.com,The US military on Friday ordered several poli...,The US military on Friday ordered several poli...,2.0,https://bit.ly/2XlTpFf,https://www.facebook.com/122163047886/posts/10...,134407,1.476190,1.0,41,10,79,3,6,0,5,11,0,0,19,13,21,6,6,7,22,11,0,3,25255,Action News Now,ActionNewsNow,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Your source for local news in the North State....,True,2009-09-01 18:48:05,en,0,25255|10157592533682887,"['Minnesota', 'US']","['Minnesota', 'US']",['Minnesota'],"['minnesota', 'us']","['Minnesota, United States', 'United States']","['minnesota', 'us']"
2,2,2,1638944876418788_2524797221166878,Facebook,2020-05-30 22:01:34,2021-03-29 08:07:35,link,Demonstrators in Jacksonville join national â...,newsbreak.com,"JACKSONVILLE, Fla. â People in Jacksonville ...",Demonstrators in Jacksonville join national â...,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/1638944876418788/post...,0,-0.800000,1.0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,1,12535316,"Jacksonville, FL Local - News Break",JacksonvilleFLLocal,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-20 18:38:30,en,0,12535316|2524797221166878,['Jacksonville'],"['Jacksonville', 'Minneapolis', 'JACKSONVILLE'...",['Jacksonville'],"['jacksonville', 'minneapolis', 'jacksonville'...","['Jacksonville, Duval County, Florida, United ...","['jacksonville', 'minneapolis', 'jacksonville'..."
3,3,3,599127280248752_1531728960321908,Facebook,2020-05-30 22:01:34,2021-07-12 09:44:28,link,Comments From Chief Newlan On Current Events |...,newsbreak.com,Chief Newlan provides a brief comment related ...,Comments From Chief Newlan On Current Events,1.0,https://www.newsbreakapp.com/m/articles?nb_red...,https://www.facebook.com/599127280248752/posts...,2851,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,2,1,2,1,2,1,1,1,0,0,5970175,"Fort Myers, FL Local - News Break",FortMyersNews,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Keep updated w/ the most interesting stories a...,False,2016-07-19 19:11:22,en,0,5970175|1531728960321908,[],[],[],[],[],[]
4,4,4,1670764379911054_2649530485367767,Facebook,2020-05-30 22:01:30,2021-05-22 23:16:40,link,Thousands Show Suppor

In [15]:
pd.set_option('display.max_rows', None)

In [52]:
ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [53]:
ct_mapping

,Unnamed: 0,City,Number,Mapping,Latitude,Longtitude,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,0,minneapolis,8686,"Minneapolis, Hennepin County, Minnesota, Unite...",44.9772995,-93.2654692,NaN,NaN,NaN,NaN
1,1,minnesota,2261,"Minnesota, United States",45.9896587,-94.6113288,NaN,NaN,NaN,NaN
2,2,atlanta,1546,"Atlanta, Fulton County, Georgia, United States",33.7489924,-84.3902644,NaN,NaN,NaN,NaN
3,3,america,1381,"America, Horst aan de Maas, Limburg, Nederland",51.4371483,5.9799001,NaN,NaN,NaN,NaN
4,4,u.s.,1071,"Unterer See, Kirchheim unter Teck, Vereinbarte...",48.63057945,9.41754455,NaN,NaN,NaN,NaN
5,5,us,863,United States,39.7837304,-100.445882,NaN,NaN,NaN,NaN
6,6,the united states,609,"The Summit, Guadalupe County, Texas, United St...",29.6330078,-97.98859404,NaN,NaN,NaN,NaN
7,7,houston,447,"Houston, Harris County, Texas, United States",29.7589382,-95.3676974,NaN,NaN,NaN,NaN
8,8,washington,365,"Washington, District of Columbia, United States",38.8950368,-77.0365427,NaN,NaN,NaN,NaN
9,9,chicago,363,"Chicago, Cook County, Illinois, United States",41.8755616,-87.6244212,NaN,NaN,NaN,NaN


In [21]:
country_list=pd.read_csv('Country.txt')

In [28]:
country_list['Name']=country_list['Name'].str.lower()

In [ ]:
country_list['Name']

In [19]:
states_list=pd.read_csv('states.csv')

In [ ]:
states_list['State']

In [45]:
from numpy.core.numeric import NaN

In [54]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['City'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [55]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(states_list['State'])):
    if ct_mapping['Mapping'][i]==states_list['State'][k]:
      ct_mapping['Mapping'][i]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [58]:
for i in range(len(ct_mapping['City'])):
  for k in range(len(country_list['Name'])):
    if ct_mapping['Mapping'][i]==country_list['Name'][k]:
      ct_mapping['Mapping'][i]=NaN

In [56]:
ct_mapping['Mapping'][4]=NaN
ct_mapping['Mapping'][5]=NaN
ct_mapping['Mapping'][6]=NaN
ct_mapping['Mapping'][7]=NaN

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [ ]:
ct_mapping

In [70]:
ct_mapping.to_csv('dataframe_mapping_0530.csv')

In [71]:
ct_mapping=pd.read_csv('dataframe_mapping_0530.csv',encoding='ISO-8859-1')

In [ ]:
ct_mapping

In [ ]:
df_post[:100]

In [ ]:
for k in range(len(df_post['NER-2'])):
  list=[]
  for i in df_post['NER-2'][k]:
    print(i)
    for j in range(len(ct_mapping['City'])):
      #print("   ")
      #print(ct_mapping['City'][j])
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  df_post['NER-2'][k]=list